In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#import lib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from scipy import spatial

import nltk
from nltk.corpus import stopwords
from nltk.corpus import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import pickle
from collections import defaultdict

#Load Data

In [ ]:
# Load the training data from a file
train = pd.read_csv('/content/drive/MyDrive/chatbot_data/dreaddit-train.csv')
test = pd.read_csv('/content/drive/MyDrive/chatbot_data/dreaddit-test.csv')
# Access the 'subreddit' column
subred = train['subreddit']
# returns the number of elements in the 'text' column of the DataFrame 'train_df'
print(len(train['text']))
# Print an array or list of unique values from the 'subred' column
subred.unique()

2838


array(['ptsd', 'assistance', 'relationships', 'survivorsofabuse',
       'domesticviolence', 'anxiety', 'homeless', 'stress',
       'almosthomeless', 'food_pantry'], dtype=object)

In [ ]:
print(test['text'][10])

I've always hated nail files. Somehow that's a part of this. God. I'm confused by it all. It's a feeling to recall it that I've carried my whole life but never understood like a cloud.


#Preprocess Data

##Tokenization, Removal of punctuations

In [ ]:
#Using Regexp from the nltk.tokenizer
tokenizer = RegexpTokenizer(r'[a-zA-Z]{2,}') #remove number and words has only one character
#The tokenization process is applied to the 'text' column of both the 'train' and 'test' dataframes using the apply() function
train['processed_text'] = train['text'].apply(lambda x: tokenizer.tokenize(x.lower()))
test['processed_text'] = test['text'].apply(lambda x: tokenizer.tokenize(x.lower()))
#'processed_text' is a new column that will be created in the train DataFrame
#lower() makes it lowercase

#Processed_text is a new column that is being added to the train DataFrame.

##Filtering stopwords

In [ ]:
def filtering_stopwords(data): #function that takes as input 'data'
  words = [w for w in data if not w in stopwords.words('english')]
  return words

# w is a variable that represents an individual word or token from the data list
#The 'english' parameter is used to specify the language for which we want to retrieve the stopwords.

In [ ]:
train['processed_text'] = train['processed_text'].apply(lambda x: filtering_stopwords(x))
test['processed_text'] = test['processed_text'].apply(lambda x: filtering_stopwords(x))

#x represents each element (text entry) in the text column of the train DataFrame
#or x represents each individual element or value in the 'processed_text' column of the DataFrame.
#It is a placeholder variable that represents the input to the lambda function,
# which in this case is each element of the 'processed_text' column

In [ ]:
train['processed_text']
# is a column in the train DataFrame.
#It represents the processed text data after applying various text preprocessing steps
#such as tokenization, lowercase conversion, stop word removal, and stemming.
#Each element in the processed_text column corresponds to a list of processed words or
#tokens for the corresponding text entry in the original text column of the train DataFrame.

0       [said, felt, way, suggeted, go, rest, trigger,...
1       [hey, assistance, sure, right, place, post, go...
2       [mom, hit, newspaper, shocked, would, knows, l...
3       [met, new, boyfriend, amazing, kind, sweet, go...
4       [october, domestic, violence, awareness, month...
                              ...                        
2833    [week, ago, precious, ignored, jan, happy, yea...
2834    [ability, cope, anymore, trying, lot, things, ...
2835    [case, first, time, reading, post, looking, pe...
2836    [find, normal, good, relationship, main, probl...
2837    [talking, mom, morning, said, sister, trauma, ...
Name: processed_text, Length: 2838, dtype: object

In [ ]:
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(data):
  lemma_text = [lemmatizer.lemmatize(i) for i in data]
  return lemma_text

In [ ]:
train['processed_text'] = train['processed_text'].apply(lambda x: word_lemmatizer(x))
test['processed_text'] = test['processed_text'].apply(lambda x: word_lemmatizer(x))

In [ ]:
train['processed_text']

0       [said, felt, way, suggeted, go, rest, trigger,...
1       [hey, assistance, sure, right, place, post, go...
2       [mom, hit, newspaper, shocked, would, know, li...
3       [met, new, boyfriend, amazing, kind, sweet, go...
4       [october, domestic, violence, awareness, month...
                              ...                        
2833    [week, ago, precious, ignored, jan, happy, yea...
2834    [ability, cope, anymore, trying, lot, thing, t...
2835    [case, first, time, reading, post, looking, pe...
2836    [find, normal, good, relationship, main, probl...
2837    [talking, mom, morning, said, sister, trauma, ...
Name: processed_text, Length: 2838, dtype: object

#Word Embedding

In [ ]:
# Load pre-trained GloVe embeddings
word2vec_output_file = '/content/drive/MyDrive/chatbot_data/glove.6B.100d.txt'
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# glove_file = 'glove.6B.100d.txt'  # path to the GloVe embeddings file
word2vec_file = '/content/drive/MyDrive/chatbot_data/glove.6B.100d.word2vec'  # output path for the converted Word2Vec file

# Convert GloVe file to Word2Vec format
glove2word2vec(word2vec_output_file, word2vec_file)

# Load the Word2Vec model
# model = KeyedVectors.load_word2vec_format(word2vec_file, binary=False)

# word2vec_output_file = 'glove.6B.100d.txt'+'.word2vec'
# Load pre-trained GloVe embeddings
smodel = KeyedVectors.load_word2vec_format(word2vec_file, binary=False)
# the glove2word2vec() function to convert the GloVe word embedding file located at '/content/drive/MyDrive/glove.6B.100d.txt'
# into the Word2Vec format and save it to the word2vec_output_file path, which is set to '/content/drive/MyDrive/glove.6B.100d.txt'.
# The converted Word2Vec file will have the same content as the original GloVe file, but it will be in the Word2Vec format,
# which can be loaded and used with the KeyedVectors.load_word2vec_format() method.

#KeyedVectors.load_word2vec_format() function in Gensim allows you to represent words as vectors.
#Word embeddings capture the semantic relationships between words by mapping them to dense vector representations in a high-dimensional space.

<ipython-input-21-301d8274eaca>:10: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(word2vec_output_file, word2vec_file)


In [ ]:
keys = ['abuse', 'anxiety', 'financial', 'ptsd', 'social', 'stress', 'love', 'friendship']
for i in keys:
  print('Vector of:', i ,smodel.get_vector(i))
  result = smodel.most_similar(positive=i, topn=10)
  print('\n10 most similar words to: ',i,result)

Vector of: abuse [ 0.6747     0.31766    0.19427   -0.23009   -0.7347     1.4631
 -0.6046    -0.098757   0.054897   0.23114   -0.074416   0.088503
 -0.26545    1.1766     0.32394    0.31836    0.073733  -0.019944
 -1.5462     0.44651   -0.26256   -0.26111   -0.32883    0.60584
 -0.66182    0.028531   0.61704   -0.34273   -0.20538    0.77077
  1.0128    -0.14495    0.0048883  0.63247   -0.26677   -0.19595
 -0.44454   -0.3079    -0.21728    0.38524   -0.83284    0.18319
 -0.083784  -0.66435   -0.13656    0.26515    0.34094   -0.27263
  0.0086811 -0.76744    0.83997   -0.44614   -0.71315    1.2288
  0.16899   -1.0177    -0.16239   -0.90029    1.3657     0.17637
  0.051633  -0.068679   0.16901   -0.95653    0.55789   -0.12813
  0.95465   -1.1387     0.61918    0.20519    0.18888    0.029914
 -0.68286    0.28326    0.36491    0.071344  -0.23957    0.32145
 -1.1288     0.39188    0.83201    0.45375    0.59285    0.50722
 -2.0063    -0.72023   -0.12334    0.24201   -0.24597   -0.65065
 -0.397

#TF-IDF Weighted Average

In [ ]:
class TfidfEmbeddingVectorizer(object):

    def __init__(self, word_model):
        self.word_model = word_model
         #word_model represents the pre-trained word embedding model
        self.word_idf_weight = None
        #This attribute will later store the IDF (Inverse Document Frequency) weights of the words.
        self.vector_size = word_model.vector_size
        #The vector_size attribute is set to the dimensionality of the word vectors in the word_model

    def default_idf(self):
        return self.max_idf

    def fit(self, docs): # comply with scikit-learn transformer requirement
      """
      Fit in a list of docs, which had been preprocessed and tokenized,
      such as word bi-grammed, stop-words removed, lemmatized, part of speech filtered.
      Then build up a tfidf model to compute each word's idf as its weight.
      Noted that tf weight is already involved when constructing average word vectors, and thus omitted.
      :param: pre_processed_docs: list of docs, which are tokenized
      :return: self
      """

      text_docs = []
      for doc in docs:
          text_docs.append(" ".join(doc))

      tfidf = TfidfVectorizer()  # default 1-gram
      tfidf.fit(text_docs)

      self.max_idf = max(tfidf.idf_)
      self.word_idf_weight = defaultdict(self.default_idf)

      for word, i in tfidf.vocabulary_.items():
          self.word_idf_weight[word] = tfidf.idf_[i]

      return self


    def transform(self, docs):       # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector

    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """

        mean = []
        for word in sent:
            if word in self.word_model.key_to_index:
                vector = self.word_model.get_vector(word)
                mean.append(vector * self.word_idf_weight[word])  # idf weighted

        if not mean:
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean

    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])


    def save(self, filename):
        """
        Save the vectorizer to a file using pickle.
        :param filename: Name of the file to save the vectorizer.
        """
        with open(filename, 'wb') as f:
            pickle.dump(self, f)


    @classmethod
    def load(cls, filename):
        """
        Load the vectorizer from a file.
        :param filename: Name of the file to load the vectorizer from.
        :return: An instance of the TfidfEmbeddingVectorizer class.
        """
        with open(filename, 'rb') as f:
            return pickle.load(f)



#In summary, this class combines the TF-IDF vectorization technique with pre-trained word embeddings
#to transform preprocessed and tokenized documents into average word vector representations that are weighted by IDF.
#The resulting vectors can be used as input features for various machine learning models or downstream tasks.


In [ ]:
tfidf_vec_tr = TfidfEmbeddingVectorizer(smodel)
tfidf_vec_tr = tfidf_vec_tr.fit(train['processed_text'])  # fit tfidf model first
X_train = tfidf_vec_tr.transform(train['processed_text'])
y_train = train['label']

# tfidf_vec_tr.fit(test['processed_text'])
X_test = tfidf_vec_tr.transform(test['processed_text'])
y_test = test['label']

In [ ]:
# Save the vectorizer
tfidf_vec_tr.save('/content/drive/MyDrive/chatbot_data/tfidf_vec_tr.pkl')

In [ ]:
# Save the weights
# Save the weights to a file
with open('/content/drive/MyDrive/chatbot_data/word_idf_weights.pkl', 'wb') as file:
    pickle.dump(tfidf_vec_tr.word_idf_weight, file)

In [ ]:
print(tfidf_vec_tr.word_idf_weight)

[[-0.847684    0.68693095  0.45550036 ...  0.14931832  0.7828862
   0.9441152 ]
 [ 0.13888466  1.6395565   1.0083388  ... -0.29444835  2.1498945
   0.60911167]
 [ 0.14925882  0.7516254   1.1579771  ...  0.20874952  1.1069926
   0.11873502]
 ...
 [-0.11177807  0.14724047  1.0613122  ... -0.16153254  1.2265683
   0.6959015 ]
 [-0.8360129   1.3445976   1.4285393  ... -1.4185493   1.7372129
   0.7974449 ]
 [-0.02265701  0.6556465   2.171653   ... -0.0295425   1.7845883
   0.17743158]]


#Train and Save the Model

In [ ]:
clf_dict = {'log reg': LogisticRegression(random_state=42),
            'naive bayes': GaussianNB(),
            'linear svc': LinearSVC(random_state=42),
            'sgd classifier': SGDClassifier(random_state=42),
            'ada boost': AdaBoostClassifier(n_estimators=100, random_state=42),
            'gradient boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
            'CART': DecisionTreeClassifier(random_state=42),
            'random forest': RandomForestClassifier(n_estimators=100, random_state=42)}
for name, clf in clf_dict.items():
    pred = clf.fit(X_train, y_train)
    y_pred = pred.predict(X_test)
    print('Accuracy of {}:'.format(name), accuracy_score(y_pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of log reg: 0.6951048951048951
Accuracy of naive bayes: 0.6363636363636364


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy of linear svc: 0.6853146853146853
Accuracy of sgd classifier: 0.5832167832167832
Accuracy of ada boost: 0.6811188811188811
Accuracy of gradient boosting: 0.7132867132867133
Accuracy of CART: 0.5986013986013986
Accuracy of random forest: 0.6951048951048951


In [ ]:
import pickle

class MajorityVoteClassifier(object):
    def __init__(self, classifiers):
        self.classifiers = classifiers

    def fit(self, X, y):
        for clf in self.classifiers:
            clf.fit(X, y)

    def predict(self, X):
        predictions = []
        for clf in self.classifiers:
            predictions.append(clf.predict(X))

        # Perform majority vote
        majority_vote = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)
        return majority_vote

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load(cls, filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)


In [ ]:
classifiers = [LogisticRegression(random_state=42),
               GaussianNB(), LinearSVC(random_state=42), SGDClassifier(random_state=42), AdaBoostClassifier(n_estimators=100, random_state=42),
               GradientBoostingClassifier(n_estimators=100, random_state=42), DecisionTreeClassifier(random_state=42), RandomForestClassifier(n_estimators=100, random_state=42)]

for name, clf in clf_dict.items(): # Replace with your list of classifiers
  mv_classifier = MajorityVoteClassifier(classifiers)
  mv_classifier.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:
# Save the combined model
mv_classifier.save('/content/drive/MyDrive/chatbot_data/majority_vote_model.pkl')